Importing the Required Packages

In [1]:
import numpy as np
import pandas as pd

Loading the dataset into the notebooks

In [2]:
# Loading the dataset
dataset = pd.read_csv("Data/new_retail_data.csv")

# Printing the top 5 rows of the dataset
dataset.head()

,Transaction_ID,Customer_ID,Name,Email,Phone,Address,City,State,Zipcode,Country,...,Total_Amount,Product_Category,Product_Brand,Product_Type,Feedback,Shipping_Method,Payment_Method,Order_Status,Ratings,products
0,8691788.0,37249.0,Michelle Harrington,Ebony39@gmail.com,1.414787e+09,3959 Amanda Burgs,Dortmund,Berlin,77985.0,Germany,...,324.086270,Clothing,Nike,Shorts,Excellent,Same-Day,Debit Card,Shipped,5.0,Cycling shorts
1,2174773.0,69749.0,Kelsey Hill,Mark36@gmail.com,6.852900e+09,82072 Dawn Centers,Nottingham,England,99071.0,UK,...,806.707815,Electronics,Samsung,Tablet,Excellent,Standard,Credit Card,Processing,4.0,Lenovo Tab
2,6679610.0,30192.0,Scott Jensen,Shane85@gmail.com,8.362160e+09,4133 Young Canyon,Geelong,New South Wales,75929.0,Australia,...,1063.432799,Books,Penguin Books,Children's,Average,Same-Day,Credit Card,Processing,2.0,Sports equipment
3,7232460.0,62101.0,Joseph Miller,Mary34@gmail.com,2.776752e+09,8148 Thomas Creek Suite 100,Edmonton,Ontario,88420.0,Canada,...,2466.854021,Home Decor,Home Depot,Tools,Excellent,Standard,PayPal,Processing,4.0,Utility knife
4,4983775.0,27901.0,Debra Coleman,Charles30@gmail.com,9.098268e+09,5813 Lori Ports Suite 269,Bristol,England,48704.0,UK,...,248.553049,Grocery,Nestle,Chocolate,Bad,Standard,Cash,Shipped,1.0,Chocolate cookies


In [3]:
init_shape = dataset.shape
print(f"Shape of the dataset: {init_shape}")

Shape of the dataset: (302010, 30)


# Data Cleaning and Preprocessing

1. Identification and Handling of Missing values

In [4]:
# Number of missing values in each column
print(f"Number of missing values in the dataset: \n{dataset.isnull().sum()}")

Number of missing values in the dataset: 
Transaction_ID      333
Customer_ID         308
Name                382
Email               347
Phone               362
Address             315
City                248
State               281
Zipcode             340
Country             271
Age                 173
Gender              317
Income              290
Customer_Segment    215
Date                359
Year                350
Month               273
Time                350
Total_Purchases     361
Amount              357
Total_Amount        350
Product_Category    283
Product_Brand       281
Product_Type          0
Feedback            184
Shipping_Method     337
Payment_Method      297
Order_Status        235
Ratings             184
products              0
dtype: int64


In [5]:
# Handling the missing values according to their data types and their Distribution

# Rows that needed to be dropped
drop_rows = ["Transaction_ID", "Customer_ID", "Name",
             "Email", "Phone", "Address", "Date", "Year",
             "Month", "Time", "Product_Brand", "Order_Status"]
for col in drop_rows:
    dataset = dataset[dataset[col].notna()]

# Categorical Columns  
mode_col = ["City", "State", "Zipcode", "Country", "Gender", "Income",
            "Customer_Segment", "Total_Purchases", "Product_Category",
            "Feedback", "Shipping_Method", "Payment_Method"]
for col in mode_col:
    dataset[col] = dataset[col].fillna(dataset[col].mode()[0])

# Continous Columns and that are sensetive to outliers
median_col = ["Age", "Amount", "Total_Amount", "Ratings"]
for col in median_col:
    dataset[col] = dataset[col].fillna(dataset[col].median())

In [6]:
print(f"Shape of the dataset after handling missing values: {dataset.shape}")

Shape of the dataset after handling missing values: (298141, 30)


In [7]:
print(f"Number of the rows that are removed: {init_shape[0] - dataset.shape[0]}")

Number of the rows that are removed: 3869


In [8]:
# Data after handling missing values
dataset.isnull().sum()

Transaction_ID      0
Customer_ID         0
Name                0
Email               0
Phone               0
Address             0
City                0
State               0
Zipcode             0
Country             0
Age                 0
Gender              0
Income              0
Customer_Segment    0
Date                0
Year                0
Month               0
Time                0
Total_Purchases     0
Amount              0
Total_Amount        0
Product_Category    0
Product_Brand       0
Product_Type        0
Feedback            0
Shipping_Method     0
Payment_Method      0
Order_Status        0
Ratings             0
products            0
dtype: int64

2. Error Correction and Inconsistencies

In [9]:
print(f"Name of the columns: {dataset.columns.values}")

Name of the columns: ['Transaction_ID' 'Customer_ID' 'Name' 'Email' 'Phone' 'Address' 'City'
 'State' 'Zipcode' 'Country' 'Age' 'Gender' 'Income' 'Customer_Segment'
 'Date' 'Year' 'Month' 'Time' 'Total_Purchases' 'Amount' 'Total_Amount'
 'Product_Category' 'Product_Brand' 'Product_Type' 'Feedback'
 'Shipping_Method' 'Payment_Method' 'Order_Status' 'Ratings' 'products']


Changing the name of few columns to make them meaningful

In [10]:
dataset.rename(columns = {
    "Total_Purchases": "Quantity",
    "Amount": "Price_per_unit"
}, inplace = True)

In [11]:
dataset.describe()

,Transaction_ID,Customer_ID,Phone,Zipcode,Age,Year,Quantity,Price_per_unit,Total_Amount,Ratings
count,2.981410e+05,298141.000000,2.981410e+05,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000,298141.000000
mean,5.494940e+06,55007.072902,5.500495e+09,50312.113359,35.478498,2023.165090,5.359431,255.194433,1367.431405,3.162618
std,2.596053e+06,26005.990411,2.595721e+09,28964.029875,15.015371,0.371262,2.866482,141.315801,1128.452562,1.320216
min,1.000007e+06,10000.000000,1.000049e+09,501.000000,18.000000,2023.000000,1.000000,10.000219,10.003750,1.000000
25%,3.246869e+06,32469.000000,3.254198e+09,25446.000000,22.000000,2023.000000,3.000000,133.032154,439.376483,2.000000
50%,5.497883e+06,55020.000000,5.504195e+09,50652.000000,32.000000,2023.000000,5.000000,255.497589,1041.252907,3.000000
75%,7.739315e+06,77506.000000,7.748878e+09,75212.000000,46.000000,2023.000000,8.000000,377.573261,2027.455133,4.000000
max,9.999995e+06,99999.000000,9.999996e+09,99949.000000,70.000000,2024.000000,10.000000,499.997911,4999.625796,5.000000


The Mean and Median of all the Numerical columns are almost identical (except: Total_Amount), it means:
- Symmetry in Data Distribution: It indicates that the data is likely symmetrically distributed around the central value.
- Lack of Skewness: it suggests that the distribution has little to no skew.
- Robustness to Outliers: It implies that there are no extreme outliers.

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298141 entries, 0 to 302009
Data columns (total 30 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Transaction_ID    298141 non-null  float64
 1   Customer_ID       298141 non-null  float64
 2   Name              298141 non-null  object 
 3   Email             298141 non-null  object 
 4   Phone             298141 non-null  float64
 5   Address           298141 non-null  object 
 6   City              298141 non-null  object 
 7   State             298141 non-null  object 
 8   Zipcode           298141 non-null  float64
 9   Country           298141 non-null  object 
 10  Age               298141 non-null  float64
 11  Gender            298141 non-null  object 
 12  Income            298141 non-null  object 
 13  Customer_Segment  298141 non-null  object 
 14  Date              298141 non-null  object 
 15  Year              298141 non-null  float64
 16  Month             298141 

Unique values in the Categorical columns

In [13]:
cat_cols = ["Gender", "Customer_Segment", "Product_Category", "Product_Brand",
            "Product_Type", "Feedback", "Shipping_Method", "Payment_Method", "Order_Status"]
for col in cat_cols:
    print(f"Unique values in {col}: {dataset[col].unique()}")
    print("-"*100)

Unique values in Gender: ['Male' 'Female']
----------------------------------------------------------------------------------------------------
Unique values in Customer_Segment: ['Regular' 'Premium' 'New']
----------------------------------------------------------------------------------------------------
Unique values in Product_Category: ['Clothing' 'Electronics' 'Books' 'Home Decor' 'Grocery']
----------------------------------------------------------------------------------------------------
Unique values in Product_Brand: ['Nike' 'Samsung' 'Penguin Books' 'Home Depot' 'Nestle' 'Apple' 'Zara'
 'Random House' 'Coca-Cola' 'Adidas' 'Pepsi' 'IKEA' 'HarperCollins'
 'Bed Bath & Beyond' 'Sony' 'Whirepool' 'Mitsubhisi' 'BlueStar']
----------------------------------------------------------------------------------------------------
Unique values in Product_Type: ['Shorts' 'Tablet' "Children's" 'Tools' 'Chocolate' 'Television' 'Shirt'
 'Decorations' 'Non-Fiction' 'Water' 'Snacks' 'T-shirt' '

In [14]:
data_rename = ["Product_Category", "Product_Brand", "Product_Type", "Payment_Method"]

for col in data_rename:
    dataset[col] = dataset[col].str.replace(" ", "_")

In [15]:
for col in cat_cols:
    print(f"Unique values in {col}: {dataset[col].unique()}")
    print("-"*100)

Unique values in Gender: ['Male' 'Female']
----------------------------------------------------------------------------------------------------
Unique values in Customer_Segment: ['Regular' 'Premium' 'New']
----------------------------------------------------------------------------------------------------
Unique values in Product_Category: ['Clothing' 'Electronics' 'Books' 'Home_Decor' 'Grocery']
----------------------------------------------------------------------------------------------------
Unique values in Product_Brand: ['Nike' 'Samsung' 'Penguin_Books' 'Home_Depot' 'Nestle' 'Apple' 'Zara'
 'Random_House' 'Coca-Cola' 'Adidas' 'Pepsi' 'IKEA' 'HarperCollins'
 'Bed_Bath_&_Beyond' 'Sony' 'Whirepool' 'Mitsubhisi' 'BlueStar']
----------------------------------------------------------------------------------------------------
Unique values in Product_Type: ['Shorts' 'Tablet' "Children's" 'Tools' 'Chocolate' 'Television' 'Shirt'
 'Decorations' 'Non-Fiction' 'Water' 'Snacks' 'T-shirt' '

Now, we will reduce the number of decimal places in a few columns.

#### Why We Do This Step
Reducing the number of decimal places can be important for several reasons:

- Readability: Fewer decimal places can make the data easier to read and interpret.
- Data Storage: Reducing the number of decimal places can slightly reduce the storage requirements of the dataset.
- Consistency: Ensuring that numerical data is presented consistently across different columns and datasets.

In [17]:
# Name of the columns that have multiple values after the decimal
round_col = ["Price_per_unit", "Total_Amount"]

# We will be rounding off the values to 2 decimals
for col in round_col:
    dataset[col] = dataset[col].round(2)

In [18]:
print("Number of unique columns in each column: ")
for col in dataset.columns:
    print(f"{col}: {len(dataset[col].unique())}")

Number of unique columns in each column: 
Transaction_ID: 291071
Customer_ID: 86641
Name: 158185
Email: 52740
Phone: 295810
Address: 295798
City: 130
State: 54
Zipcode: 93789
Country: 5
Age: 53
Gender: 2
Income: 3
Customer_Segment: 3
Date: 366
Year: 2
Month: 12
Time: 83579
Quantity: 10
Price_per_unit: 48879
Total_Amount: 191765
Product_Category: 5
Product_Brand: 18
Product_Type: 33
Feedback: 4
Shipping_Method: 3
Payment_Method: 4
Order_Status: 4
Ratings: 5
products: 318
